In [1]:
import torch
import pickle
import numpy as np
from theoretical_analysis.sparse_nnpca_analysis import H, theta, generate_nnpca_matrix

/cis/home/yma93/miniconda3/envs/nonlinear_laplacian/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("logs/gaussian_sparse_nnpca/optimization_tanh.pkl", "rb") as f:
    res = pickle.load(f)

In [4]:
N = 100
beta = 1
sigma = lambda x: res.x[0] * np.tanh(res.x[1] * x)
results = {}
for n in [2000, 3000, 5000, 10000]:
    A_p_batch = torch.zeros((N, n, n))
    y_batch = torch.zeros((N, n))

    for i in range(N):
        A_p, y = generate_nnpca_matrix(n, beta)
        A_p_batch[i] = torch.tensor(A_p)
        y_batch[i] = torch.tensor(y)

    # Compute L_p in batch
    diag_values = res.x[0] * torch.tanh(res.x[1] * A_p_batch.sum(dim=2))
    L_p_batch = A_p_batch + torch.diag_embed(diag_values)
    evals, evecs = torch.linalg.eigh(L_p_batch)
    L_p_evals = evals[:, -1].numpy()
    L_p_evals_mean = np.mean(L_p_evals)

    # Compute the theoretical prediction
    prediction = H(theta(c=beta, sigma=sigma, sigma_image=[-res.x[0], res.x[0]], tol=2e-12), sigma)
    results[n] = [L_p_evals_mean, prediction]
    print(f"n={n}, L_p_evals={L_p_evals_mean}, prediction={prediction}")

n=2000, L_p_evals=3.034599542617798, prediction=3.016802824122022
n=3000, L_p_evals=3.0099093914031982, prediction=3.016802824122022
n=5000, L_p_evals=3.019732713699341, prediction=3.016802824122022
